<a href="https://colab.research.google.com/github/WHU-Peter/COMP6200-Project/blob/main/vgg_model_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! /opt/bin/nvidia-smi

/bin/bash: /opt/bin/nvidia-smi: No such file or directory


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, models, transforms
import os
from torch.autograd import Variable
import matplotlib.pyplot as plt
import time
import math
import numpy as np
import timeit

# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(seed)

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=4,
                                          shuffle=True)
testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=1,
                                         shuffle=False)
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# baseline_model = models.vgg16(pretrained=True).to(device)  # 使用VGG16 网络预训练好的模型

# epoch = 30  #Training times
# learning_rate = 1e-5  #Learning rate

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(baseline_model.parameters(), lr=learning_rate)
# Loss_Baseline_Model = []

# print('Baseline_Model Started Training')

# for epoch in range(epoch):    #Iteration
#     running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
#         inputs, labels = data
#                  #Initialize gradient
#         optimizer.zero_grad()

#         outputs = baseline_model(inputs.to(device))
#         loss = criterion(outputs, labels.to(device))
#         loss.backward()
#         optimizer.step()

#         # Print loss
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.5f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             Loss_Baseline_Model.append(running_loss)
#             running_loss = 0.0

# print('Baseline_Model Finished Training')

In [5]:
# correct = 0
# total = 0
# baseline_model.eval()
# for data in testloader:
#     images, labels = data
#     images, labels = images.to(device), labels.to(device)
#     outputs = baseline_model(images)
#     _, predicted = torch.max(outputs.data, 1)
#     total += labels.size(0)
#     correct += (predicted == labels).sum().item()

# print('Accuracy of the network on the 10000 test images: %.2f %%' % (
#     100 * correct / total))

In [6]:
# build the model 
baseline_model = models.vgg16(pretrained=True).to(device)  # 使用VGG16 网络预训练好的模型
learning_rate = 1e-5  #Learning rate
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(baseline_model.parameters(), lr=learning_rate)

def baseline_model_train(inputs, labels):
  baseline_model.train()
  # zero the parameter gradients
  optimizer.zero_grad()
  # forward + loss + backward + optimise (update weights)
  outputs = baseline_model(inputs)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()

def baseline_model_inference(inputs):
  baseline_model.eval()
  outputs = baseline_model(inputs)

x = 0
y = 0
for data in testloader:
  inputs, labels = data
  x = inputs.to(device)
  y = labels.to(device)
  break

t_baseline_train = timeit.Timer(
    stmt='baseline_model_train(x, y)', 
    setup='from __main__ import baseline_model_train',
    globals={'x': x, 'y' : y}
)

t_baseline_inference = timeit.Timer(
    stmt='baseline_model_inference(x)', 
    setup='from __main__ import baseline_model_inference',
    globals={'x': x}
)

print(f'baseline_model_train(x, y):  {t_baseline_train.timeit(100) / 100 * 1e3:>5.1f} ms')
print(f'baseline_model_inference(x):  {t_baseline_inference.timeit(100) / 100 * 1e3:>5.1f} ms')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


baseline_model_train(x, y):  1905.3 ms
baseline_model_inference(x):   72.6 ms


In [7]:
def softmax_temperature(logits, temperature):
    pro = F.softmax(logits / temperature, dim=-1)
    return pro;

In [8]:
class LearnableLookUpTable(nn.Module):
    def __init__(self, num_embeddings: int, embedding_dim: int, init_temperature: float):
        super(LearnableLookUpTable, self).__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim)
        self.temperature = nn.Parameter(torch.tensor(init_temperature, requires_grad=True))

    def forward(self, x):
        # if self.training:
          x = softmax_temperature(x, self.temperature)
          return x @ self.emb.weight
        # else:
        #   x = softmax_temperature(x, 0.00001)
        #   x = mapping_onehot_vector(x)
        #   return self.emb(x)

In [9]:
class Learnable_VGG_LookUpTable(nn.Module):
  def __init__(self, originalModel, init_temperature):
    super(Learnable_VGG_LookUpTable, self).__init__()
    # self.features = originalModel.features
    # self.avgpool = originalModel.avgpool
    self.vgg = nn.Sequential(*list(originalModel.children())[:-1])
    # self.classifier = originalModel.classifier
    self.look_up_table = LearnableLookUpTable(25088, 4096, init_temperature)
    self.classifier = nn.Sequential(*list(originalModel.classifier.children())[1:])
    
  def forward(self, x):
    # x = self.features(x)
    # x = self.avgpool(x)
    x = self.vgg(x)
    x = x.view(x.size(0), -1)
    x = self.look_up_table(x)
    x = self.classifier(x)
    return x

In [10]:
# pre_model = models.vgg16(pretrained=True)  # 使用VGG16 网络预训练好的模型
# for parma in pre_model.parameters():  # 设置自动梯度为false
#     parma.requires_grad = True
# learnable_lut_model = Learnable_VGG_LookUpTable(pre_model, 1.).to(device)

# epoches = 30  #Training times
# learning_rate = 1e-5  #Learning rate

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(learnable_lut_model.parameters(), lr=learning_rate)
# Loss_Learnable_Model = []
# Temperature = []

# print('Learnable_lut_model Started Training')
# for epoch in range(epoches):    #Iteration
#     running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
#         inputs, labels = data
#                  #Initialize gradient
#         optimizer.zero_grad()
#         outputs = learnable_lut_model(inputs.to(device))
#         loss = criterion(outputs, labels.to(device))
#         loss.backward()
#         optimizer.step()

#         # for parameter in optimizer.param_groups[0]['params']:
#         #   print(np.any(parameter.grad.cpu().numpy()==0))

#         # Print loss
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.5f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             Loss_Learnable_Model.append(running_loss)
#             running_loss = 0.0

#             for parameter in learnable_lut_model.named_parameters():
#               if parameter[0] == 'look_up_table.temperature':
#                 print('[%d, %5d] temperature: %.5f' %
#                   (epoch + 1, i + 1, parameter[1].data.detach().cpu().numpy()))
#                 Temperature.append(parameter[1].data.detach().cpu().numpy())

# print('Learnable_lut_model Finished Training')

In [11]:
# correct = 0
# total = 0
# learnable_lut_model.eval()
# for data in testloader:
#     images, labels = data
#     images, labels = images.to(device), labels.to(device)
#     outputs = learnable_lut_model(images)
#     _, predicted = torch.max(outputs.data, 1)
#     total += labels.size(0)
#     correct += (predicted == labels).sum().item()

# print('Accuracy of the network on the 10000 test images: %.2f %%' % (
#     100 * correct / total))

In [12]:
# build the model 
pre_model = models.vgg16(pretrained=True)  # 使用VGG16 网络预训练好的模型
for parma in pre_model.parameters():  # 设置自动梯度为false
    parma.requires_grad = True
learnable_lut_model = Learnable_VGG_LookUpTable(pre_model, 1.).to(device)
learning_rate = 1e-5  #Learning rate
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(learnable_lut_model.parameters(), lr=learning_rate)

def learnable_model_train(inputs, labels):
  learnable_lut_model.train()
  # zero the parameter gradients
  optimizer.zero_grad()
  # forward + loss + backward + optimise (update weights)
  outputs = learnable_lut_model(inputs)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()

def learnable_model_inference(inputs):
  learnable_lut_model.eval()
  outputs = learnable_lut_model(inputs)

x = 0
y = 0
for data in testloader:
  inputs, labels = data
  x = inputs.to(device)
  y = labels.to(device)
  break

t_learnable_train = timeit.Timer(
    stmt='learnable_model_train(x, y)', 
    setup='from __main__ import learnable_model_train',
    globals={'x': x, 'y' : y}
)

t_learnable_inference = timeit.Timer(
    stmt='learnable_model_inference(x)', 
    setup='from __main__ import learnable_model_inference',
    globals={'x': x}
)

print(f'learnable_model_train(x, y):  {t_learnable_train.timeit(100) / 100 * 1e3:>5.1f} ms')
print(f'learnable_model_inference(x):  {t_learnable_inference.timeit(100) / 100 * 1e3:>5.1f} ms')

learnable_model_train(x, y):  1908.4 ms
learnable_model_inference(x):   66.8 ms


In [13]:
class AnnealingLookUpTable(nn.Module):
    def __init__(self, num_embeddings: int, embedding_dim: int):
        super(AnnealingLookUpTable, self).__init__()
        self.embedding_dim = embedding_dim
        self.emb = nn.Embedding(num_embeddings, embedding_dim)

    def forward(self, x, temperature):
        if self.training:
          x = softmax_temperature(x, temperature)
          return x @ self.emb.weight
        else:
          # x = softmax_temperature(x, 0.00001)
          # x = mapping_onehot_vector(x)
          # return self.emb(x)

          x = softmax_temperature(x, temperature)
          nozero = torch.nonzero(x);
          out = np.zeros((x.shape[0], self.embedding_dim))
          out = torch.tensor(out).to(device)
          # print(np.array(nozero).shape[1])
          for i in range(x.shape[0]):
            idx = torch.where(nozero[:,0]==i)[0]
            rows = nozero[idx, 1].long()
            out[i] = torch.mean(self.emb(rows), axis=0)
          return out.float()

In [14]:
class Annealing_VGG_LookUpTable(nn.Module):
  def __init__(self, originalModel):
    super(Annealing_VGG_LookUpTable, self).__init__()
    # self.features = originalModel.features
    # self.avgpool = originalModel.avgpool
    self.vgg = nn.Sequential(*list(originalModel.children())[:-1])
    # self.classifier = originalModel.classifier
    self.look_up_table = AnnealingLookUpTable(25088, 4096)
    self.classifier = nn.Sequential(*list(originalModel.classifier.children())[1:])
    
  def forward(self, x, temperature):
    # x = self.features(x)
    # x = self.avgpool(x)
    x = self.vgg(x)
    x = x.view(x.size(0), -1)
    x = self.look_up_table(x, temperature)
    x = self.classifier(x)
    return x

In [15]:
# pre_model = models.vgg16(pretrained=True)  # 使用VGG16 网络预训练好的模型
# for parma in pre_model.parameters():  # 设置自动梯度为false
#     parma.requires_grad = True
# Annealing_lut_model = Annealing_VGG_LookUpTable(pre_model).to(device)

# epoches = 30  #Training times
# learning_rate = 1e-5  #Learning rate

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(Annealing_lut_model.parameters(), lr=learning_rate)
# Loss_Annealing_Model = []
# idx = 0

# print('Loss_Annealing_Model Started Training')
# for epoch in range(epoches):    #Iteration
#     running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
#         idx = idx + 1
#         inputs, labels = data
#                  #Initialize gradient
#         optimizer.zero_grad()
#         outputs = Annealing_lut_model(inputs.to(device), torch.tensor(max(0.0001, math.exp(-1.4 * math.pow(10, -5) * idx)), device= device))
#         loss = criterion(outputs, labels.to(device))
#         loss.backward()
#         optimizer.step()

#         # for parameter in optimizer.param_groups[0]['params']:
#         #   print(np.any(parameter.grad.cpu().numpy()==0))

#         # Print loss
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.5f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             Loss_Annealing_Model.append(running_loss)
#             running_loss = 0.0

# print('Loss_Annealing_Model Finished Training')

In [16]:
# correct = 0
# total = 0
# Annealing_lut_model.eval()
# for data in testloader:
#     images, labels = data
#     images, labels = images.to(device), labels.to(device)
#     outputs = Annealing_lut_model(images, 0.0001)
#     _, predicted = torch.max(outputs.data, 1)
#     total += labels.size(0)
#     correct += (predicted == labels).sum().item()

# print('Accuracy of the network on the 10000 test images: %.2f %%' % (
#     100 * correct / total))

In [17]:
# build the model 
pre_model = models.vgg16(pretrained=True)  # 使用VGG16 网络预训练好的模型
for parma in pre_model.parameters():  # 设置自动梯度为false
    parma.requires_grad = True
Annealing_lut_model = Annealing_VGG_LookUpTable(pre_model).to(device)
learning_rate = 1e-5  #Learning rate
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Annealing_lut_model.parameters(), lr=learning_rate)

def annealing_model_train(inputs, labels):
  Annealing_lut_model.train()
  # zero the parameter gradients
  optimizer.zero_grad()
  # forward + loss + backward + optimise (update weights)
  outputs = Annealing_lut_model(inputs, 0.001)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()

def annealing_model_inference(inputs):
  Annealing_lut_model.eval()
  outputs = Annealing_lut_model(inputs, 0.001)

x = 0
y = 0
for data in testloader:
  inputs, labels = data
  x = inputs.to(device)
  y = labels.to(device)
  break

t_annealing_train = timeit.Timer(
    stmt='annealing_model_train(x, y)', 
    setup='from __main__ import annealing_model_train',
    globals={'x': x, 'y' : y}
)

t_annealing_inference = timeit.Timer(
    stmt='annealing_model_inference(x)', 
    setup='from __main__ import annealing_model_inference',
    globals={'x': x}
)

print(f'annealing_model_train(x, y):  {t_annealing_train.timeit(100) / 100 * 1e3:>5.1f} ms')
print(f'annealing_model_inference(x):  {t_annealing_inference.timeit(100) / 100 * 1e3:>5.1f} ms')

annealing_model_train(x, y):  1820.5 ms
annealing_model_inference(x):   29.7 ms
